# Demo: Bag of Words
This is an example from sklearn (http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html) of using BOW for featurizing text.

Use as toy example for featurizing inputs not natively in $\mathbf{R}^d$.

**Concept check question**: what is $\phi(X)$ (if we use count vectorizer)?

**Answer**: $\phi(X) = [\#word_{1},...,\#word_{35788}]$ sparsified) (for count vectorizer).

In [19]:
categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']

In [20]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train',
     categories=categories, shuffle=True, random_state=42)


In [21]:
len(twenty_train.data)

2257

In [22]:
print(twenty_train.data[0])

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton
Organization: The City University
Lines: 14

Does anyone know of a good way (standard PC application/PD utility) to
convert tif/img/tga files into LaserJet III format.  We would also like to
do the same, converting to HPGL (HP plotter) files.

Please email any response.

Is this the correct group?

Thanks in advance.  Michael.
-- 
Michael Collier (Programmer)                 The Computer Unit,
Email: M.P.Collier@uk.ac.city                The City University,
Tel: 071 477-8000 x3769                      London,
Fax: 071 477-8565                            EC1V 0HB.



## Bags of words
The most intuitive way to featurize text is the bags of words representation:

- assign a fixed integer id to each word occurring in any document of the training set (for instance by building a dictionary from words to integer indices).
- for each document $i$, count the number of occurrences of each word $w$ and store it in $X[i, j]$ as the value of feature $j$ where $j$ is the index of word $w$ in the dictionary
The bags of words representation implies that $n_{features}$ is the number of distinct words in the corpus: this number is typically larger than 100,000.

If $n_{samples}$ == 10000, storing $X$ as a numpy array of type float32 would require 10000 x 100000 x 4 bytes = 4GB in RAM.

Fortunately, most values in $X$ will be zeros since for a given document less than a couple thousands of distinct words will be used. For this reason we say that bags of words are typically high-dimensional sparse datasets. We can save a lot of memory by only storing the non-zero parts of the feature vectors in memory.

scipy.sparse matrices are data structures that do exactly this, and scikit-learn has built-in support for these structures.

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(2257, 35788)

In [24]:
type(X_train_counts)

scipy.sparse.csr.csr_matrix

### Note under the hood this built vocabulary mapping from words to ints.

In [25]:
count_vect.vocabulary_.get(u'algorithm')

4690

## TF-IDF

Scaling using TF-IDF gives better performance on this task -- see sklearn docs for explanation of this transformer. Note this scale the counts, but doesn't change the size of the feature space.

In [26]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(2257, 35788)

## Build our classifier

In [27]:
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
import numpy as np
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, random_state=42,
                                           max_iter=5, tol=None)),
])
text_clf.fit(twenty_train.data, twenty_train.target)  

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                ('clf',
                 SGDClassifier(alpha=0.001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                      

In [29]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)
docs_test = twenty_test.data

In [30]:
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)            

0.9101198402130493

In [31]:
from sklearn import metrics
print(metrics.classification_report(twenty_test.target, predicted,
      target_names=twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.95      0.80      0.87       319
         comp.graphics       0.87      0.98      0.92       389
               sci.med       0.94      0.89      0.91       396
soc.religion.christian       0.90      0.95      0.93       398

              accuracy                           0.91      1502
             macro avg       0.91      0.91      0.91      1502
          weighted avg       0.91      0.91      0.91      1502



In [32]:
for doc, category in zip(twenty_test.data[3:6], predicted[3:6]):
     print('%s... => %s\n\n' % (doc[0:100], twenty_train.target_names[category]))

From: livesey@solntze.wpd.sgi.com (Jon Livesey)
Subject: Re: free moral agency
Organization: sgi
Lin... => alt.atheism


From: jhpb@sarto.budd-lake.nj.us (Joseph H. Buehler)
Subject: the ancient canon of the Roman rite
Or... => soc.religion.christian


From: bil@okcforum.osrhe.edu (Bill Conner)
Subject: Re: free moral agency
Nntp-Posting-Host: okcforu... => alt.atheism


